## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]


In [4]:
# Create a list of at least 10 column names to use as X data
sel_columns = ['Education',
               'Age',
               'DistanceFromHome',
               'JobSatisfaction',
               'OverTime',
               'YearsInCurrentRole',
               'WorkLifeBalance',
               'YearsAtCompany',
               'TotalWorkingYears',
               'HourlyRate']


# Create X_df using your selected columns
X_df = attrition_df[sel_columns]

# Show the data types for X_df
X_df.dtypes


Education              int64
Age                    int64
DistanceFromHome       int64
JobSatisfaction        int64
OverTime              object
YearsInCurrentRole     int64
WorkLifeBalance        int64
YearsAtCompany         int64
TotalWorkingYears      int64
HourlyRate             int64
dtype: object

In [5]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df)


In [6]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train['OverTime'].value_counts()

from sklearn.preprocessing import OneHotEncoder
overtime_encoder = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')
overtime_encoder.fit(np.array(X_train['OverTime']).reshape(-1, 1))
X_train['OverTimeEncoded'] = overtime_encoder.transform(np.array(X_train['OverTime']).reshape(-1, 1))
X_test['OverTimeEncoded'] = overtime_encoder.transform(np.array(X_test['OverTime']).reshape(-1, 1))

X_train = X_train.drop(columns='OverTime')
X_test = X_test.drop(columns='OverTime')

print(X_train.dtypes)
print(X_test.dtypes)

Education               int64
Age                     int64
DistanceFromHome        int64
JobSatisfaction         int64
YearsInCurrentRole      int64
WorkLifeBalance         int64
YearsAtCompany          int64
TotalWorkingYears       int64
HourlyRate              int64
OverTimeEncoded       float64
dtype: object
Education               int64
Age                     int64
DistanceFromHome        int64
JobSatisfaction         int64
YearsInCurrentRole      int64
WorkLifeBalance         int64
YearsAtCompany          int64
TotalWorkingYears       int64
HourlyRate              int64
OverTimeEncoded       float64
dtype: object


In [7]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_scaler = scaler.fit(X_train)

# Scale the training and testing data

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [8]:
# Create a OneHotEncoder for the Department column
department_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop=None)

# Fit the encoder to the training data
department_encoder.fit(np.array(y_train['Department']).reshape(-1, 1))

# Create two new variables by applying the encoder
# to the training and testing data
y_department_train = department_encoder.transform(np.array(y_train['Department']).reshape(-1, 1))
y_department_test = department_encoder.transform(np.array(y_test['Department']).reshape(-1, 1))

y_department_train



array([[0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [9]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop=None)

# Fit the encoder to the training data
attrition_encoder.fit(np.array(y_train['Attrition']).reshape(-1, 1))

# Create two new variables by applying the encoder
# to the training and testing data
y_attrition_train = attrition_encoder.transform(np.array(y_train['Attrition']).reshape(-1, 1))
y_attrition_test = attrition_encoder.transform(np.array(y_test['Attrition']).reshape(-1, 1))

y_attrition_train


array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

## Create, Compile, and Train the Model

In [10]:
# Find the number of columns in the X training data
number_input_features = len(X_train.columns)

# Create the input layer
input = layers.Input(number_input_features, name='input')

# Create at least two shared layers
shared1 = layers.Dense(64, activation='relu', name='shared1')(input)
shared2 = layers.Dense(128, activation='relu', name='shared2')(shared1)

In [11]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden = layers.Dense(32, activation='relu', name='department_hidden')(shared2)

# Create the output layer
department_output = layers.Dense(y_department_train.shape[1], activation='sigmoid', name='department_output')(department_hidden)


In [12]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(32, activation='relu', name='attrition_hidden')(shared2)

# Create the output layer
attrition_output = layers.Dense(y_attrition_train.shape[1], activation='sigmoid', name='attrition_output')(attrition_hidden)


In [13]:
# Create the model
model = Model(inputs=input, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={
                  'department_output': 'categorical_crossentropy',
                  'attrition_output': 'binary_crossentropy'
              },
              metrics={
                  'department_output': 'accuracy',
                  'attrition_output': 'accuracy'
              })


# Summarize the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input (InputLayer)          [(None, 10)]                 0         []                            
                                                                                                  
 shared1 (Dense)             (None, 64)                   704       ['input[0][0]']               
                                                                                                  
 shared2 (Dense)             (None, 128)                  8320      ['shared1[0][0]']             
                                                                                                  
 department_hidden (Dense)   (None, 32)                   4128      ['shared2[0][0]']             
                                                                                              

In [14]:
# Train the model
fit_model = model.fit(X_train_scaled, {'department_output': y_department_train, 'attrition_output': y_attrition_train}, epochs=50)


Epoch 1/50
35/35 [==============================] - 5s 6ms/step - loss: 1.3912 - department_output_loss: 0.8583 - attrition_output_loss: 0.5329 - department_output_accuracy: 0.6080 - attrition_output_accuracy: 0.8013
Epoch 2/50
35/35 [==============================] - 0s 7ms/step - loss: 1.1883 - department_output_loss: 0.7770 - attrition_output_loss: 0.4114 - department_output_accuracy: 0.6515 - attrition_output_accuracy: 0.8448
Epoch 3/50
35/35 [==============================] - 0s 5ms/step - loss: 1.1533 - department_output_loss: 0.7667 - attrition_output_loss: 0.3867 - department_output_accuracy: 0.6506 - attrition_output_accuracy: 0.8448
Epoch 4/50
35/35 [==============================] - 0s 4ms/step - loss: 1.1364 - department_output_loss: 0.7610 - attrition_output_loss: 0.3754 - department_output_accuracy: 0.6525 - attrition_output_accuracy: 0.8503
Epoch 5/50
35/35 [==============================] - 0s 5ms/step - loss: 1.1203 - department_output_loss: 0.7535 - attrition_output_l

In [15]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test_scaled, {'department_output': y_department_test, 'attrition_output': y_attrition_test})
test_results

12/12 [==============================] - 0s 3ms/step - loss: 2.1875 - department_output_loss: 1.3384 - attrition_output_loss: 0.8491 - department_output_accuracy: 0.5978 - attrition_output_accuracy: 0.8288


[2.1874947547912598,
 1.3383820056915283,
 0.849112868309021,
 0.5978260636329651,
 0.8288043737411499]

In [16]:
# Print the accuracy for both department and attrition
print(f"Department predictions accuracy: {test_results[3]}")
print(f"Attrition predictions accuracy: {test_results[4]}")

Department predictions accuracy: 0.5978260636329651
Attrition predictions accuracy: 0.8288043737411499


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy does not seem to be a good measure here as its high for one and not the other, could be due to the imbalanced data
2. Sigmoid activation function works for both
3. Adding more training data